In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
import tensorflow_hub as hub

In [5]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/5"
model = hub.load(module_url)

ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\prabh\AppData\Local\Temp\tfhub_modules\063d866c06683311b44b4992fd46003be952409c' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

In [4]:
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from textblob import TextBlob

def tokenization(text):
    # REMOVING PUNCTUATIONS
    inputText = re.sub(r'[^\w\s]','',text)
    inputText = re.sub(r'\d+', '', inputText)
    lowerString = inputText.lower()
    wordToknized = word_tokenize(lowerString)
    return wordToknized

def removeStopwords(tokens):
    wordList = stopwords.words('english')
    filtered_words = [word for word in tokens if word not in wordList]
    
    return filtered_words

def wordlemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    noun = ['NN', 'NNS', 'NNP', 'NNPS']
    verb = ['VB','VBD','VBG','VBN','VBP','VBZ']
    adverb = ['RB', 'RBR', 'RBS' ]
    adjective = ['JJ','JJR','JJS']
    for word in tokens:
        postTag = dict(pos_tag([word]))
        if postTag[word] in noun:
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='n'))
        elif postTag[word] in verb:
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='v'))
        elif postTag[word] in adjective:
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='a'))
        elif postTag[word] in adverb:
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            lemmatized_words.append(lemmatizer.lemmatize(word))
    
    return lemmatized_words

def correct_spelling(text):
    blob = TextBlob(text)
    corrected_text = str(blob.correct())
    return corrected_text

In [10]:
df = pd.read_csv('Movie-2023-06-15.csv')[['description','name']]
description = list(df['description'])

description[0]

'An orphaned boy enrolls in a school of wizardry, where he learns the truth about himself, his family and the terrible evil that haunts the magical world.\nLord Voldemort, an evil and powerful dark wizard, has just been defeated. When he tried to kill a one-year-old boy, Harry Potter (Daniel Radcliffe), the killing curse rebounded upon him, destroying his body. Harry is left an orphan with a lightning-bolt scar on his forehead, Voldemort having killed his parents, Lily (Geraldine Somerville) and James (Adrian Rawlins) Potter. Professors Dumbledore (Richard Harris) and McGonagall (Dame Maggie Smith) and Gamekeeper Hagrid (Robbie Coltrane) leave him on the doorstep of his ultra-conventional, insensitive, negligent Muggle (non-magical) relatives, the Dursley family, who take him in. Harry\'s relatives decide to conceal his magical heritage from him and make him live in a cupboard under the stairs for ten years.Shortly before Harry\'s eleventh birthday, he receives a letter addressed speci

In [15]:
pipeline_functions = [correct_spelling, tokenization, removeStopwords, wordlemmatize]
final_synopsis =[]
for i in range(len(description)):
    processed_data = description[i]
    for func in pipeline_functions:
        processed_data = func(processed_data)

    final_result = processed_data
    final_text = ' '.join(final_result)
    final_synopsis.append(final_text)
#print('Final Processed Text',final_result)

In [14]:
final_text = ' '.join(final_result)
print(final_text)

orphan boy enrol school wizard learns truth family terrible evil taunt magical world word voldemort evil powerful dark wizard defeat try kill oneyearold boy carry daniel radcliffe kill curse resound upon destroy body carry left orphan lightningbolt scar forehead voldemort kill parent oily geraldine somerville james adrian rawlins professor dumbledore richard harris mcgonagall maggie smith gamekeeper madrid hobby coltrane leave footstep ultraconventional insensitive negligent jungle nonmagical relative horsley family take carry relative decide conceal magical heritage make live cupboard stair ten yearsshortly carry eleventh birthday receives letter address specifically outrage uncle however read burn carry chance look content tender give dursleys receive successively large number correspondence soon uncle vernon richard griffith becomes paranoid dursleys carry tow hide hut small island escape night happens carry birthday visit enormous man name madrid burst lock door hut madrid hold dur

In [1]:
description

NameError: name 'description' is not defined

In [4]:
embedding = model(description)
embedding

<tf.Tensor: shape=(89, 512), dtype=float32, numpy=
array([[-0.00384051,  0.0426361 ,  0.0219461 , ...,  0.04996588,
         0.05043023, -0.0434497 ],
       [-0.01248753,  0.03504341, -0.04612051, ...,  0.04951605,
         0.05161136, -0.05100219],
       [ 0.0324989 ,  0.00092554, -0.04940296, ...,  0.04821357,
         0.04940806, -0.04940758],
       ...,
       [ 0.0397782 , -0.02828955, -0.03873897, ..., -0.03066262,
         0.05211931, -0.05258561],
       [ 0.04527739,  0.04157003,  0.03557713, ...,  0.05006167,
         0.05043361,  0.05042665],
       [ 0.04327534,  0.03478754,  0.05392738, ..., -0.0504896 ,
         0.0555308 , -0.05007016]], dtype=float32)>

In [5]:
nn = NearestNeighbors(n_neighbors=1)
nn.fit(embedding)

NearestNeighbors(n_neighbors=1)

In [6]:
def detectmovie(descriptionText):
    emd = model([descriptionText])
    neighbours = nn.kneighbors(emd,return_distance=False)[0]
    return df['name'].iloc[neighbours].tolist()

In [8]:
detectmovie('magic')

["Harry Potter and the Sorcerer's Stone"]

In [1]:
import joblib 
joblib.dump(nn,'searchMovie.sav')

NameError: name 'nn' is not defined